In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
FRA_df=pd.read_csv(r'C:\Users\M Sayan\OneDrive\Documents\FRA Project\Company_FRA.csv')

In [ ]:
FRA_df.head()

In [ ]:
FRA_df.columns=FRA_df.columns.str.replace('as % of','to')
FRA_df.columns=FRA_df.columns.str.replace(' ','_').str.replace('/','by').str.replace('%','Percentage').str.replace('(','in_').str.replace(')','')

In [ ]:
list(FRA_df.columns)

In [ ]:
FRA_df.drop('Num',axis=1,inplace=True)

In [ ]:
FRA_df.info()

In [ ]:
FRA_df.isnull().sum()

In [ ]:
FRA_df.describe()

In [ ]:
FRA_df.shape

In [ ]:
FRA_df.duplicated().sum()

In [ ]:
FRA_df.drop_duplicates(inplace=True)

In [ ]:
FRA_df.duplicated().sum()

In [ ]:
def remove_outlier(col):
    sorted(col)
    Q1,Q3=np.percentile(col,[25,75])
    IQR=Q3-Q1
    lower_range= Q1-(1.5 * IQR)
    upper_range= Q3+(1.5 * IQR)
    return lower_range, upper_range

In [ ]:
Lower_Outliers=[]
Upper_Outliers=[]
for column in FRA_df.columns:
    lr,ur=remove_outlier(FRA_df[column])
    Lower_Outliers.append(len(list(np.where(FRA_df[column]<lr)[0])))
    Upper_Outliers.append(len(list(np.where(FRA_df[column]>ur)[0])))

In [ ]:
Outliers_df=pd.DataFrame({'Lower_Outliers':Lower_Outliers,'Upper_Outliers':Upper_Outliers},index=FRA_df.columns)

In [ ]:
Outliers_df

In [ ]:
Outliers_df[(Outliers_df.Lower_Outliers>0) | (Outliers_df.Upper_Outliers>0)]

In [ ]:
FRA_df.boxplot(['Networth_Next_Year','Total_assets','Net_worth','PAT_to_net_worth','Shareholders_funds','Capital_employed','Debt_to_equity_ratio_in_times'],figsize=(20,10))
plt.show()

In [ ]:
for column in FRA_df.columns:
    p_1,p_2=np.percentile(FRA_df[column],[5,95])
    FRA_df[column]=np.where(FRA_df[column]>p_2,p_2,FRA_df[column])
    FRA_df[column]=np.where(FRA_df[column]<p_1,p_1,FRA_df[column])

In [ ]:
FRA_df.boxplot(['Networth_Next_Year','Total_assets','Net_worth','PAT_to_net_worth','Shareholders_funds','Capital_employed','Debt_to_equity_ratio_in_times'],figsize=(20,10))
plt.show()

In [ ]:
Missing_df=pd.DataFrame(FRA_df.isnull().sum(),columns=['Missing_Value'])
Missing_df

In [ ]:
Missing_df[Missing_df.Missing_Value > 0]

In [ ]:
for column in FRA_df.columns:
    FRA_df[column]=FRA_df[column].fillna(FRA_df[column].median())

In [ ]:
FRA_df.isnull().sum()

In [ ]:
FRA_df.head()

In [ ]:
FRA_df['Default']=np.where(FRA_df['Networth_Next_Year'] > 0,0,1)

In [ ]:
FRA_df['Default'].value_counts(normalize=True)

In [ ]:
df_1=pd.DataFrame(FRA_df[FRA_df['Default']==0].median(),columns=['0'])
df_1

In [ ]:
df_2=pd.DataFrame(FRA_df[FRA_df['Default']==1].median(),columns=['1'])
df_2

In [ ]:
median_df=pd.concat([df_1,df_2],axis=1)

In [ ]:
median_df[median_df['0']!=median_df['1']]

In [ ]:
median_df[median_df['0']!=median_df['1']].shape

In [ ]:
FRA_df.head()

In [ ]:
sns.boxplot(data=FRA_df,x='Default',y='Networth_Next_Year')
plt.title('Boxplot of Networth Next Year')
plt.show()

In [ ]:
sns.boxplot(data=FRA_df,x='Default',y='Total_assets')
plt.title('Boxplot of Total assets')
plt.show()

In [ ]:
sns.boxplot(data=FRA_df,x='Default',y='Total_liabilities')
plt.title('Boxplot of Total liabilities')
plt.show()

In [ ]:
sns.boxplot(data=FRA_df,x='Default',y='Shareholders_funds')
plt.title('Boxplot of Shareholders funds')
plt.show()

In [ ]:
sns.barplot(data=FRA_df,x='Default',y='PAT_to_net_worth')
plt.title('Barplot of PAT to net worth')
plt.show()

In [ ]:
FRA_df.head()

In [ ]:
FRA_df['PAT_to_net_worth'].groupby(FRA_df['Default']).mean()

In [ ]:
sns.barplot(data=FRA_df,x='Default',y='Contingent_liabilities_by_Net_worth_in_Percentage')
plt.title('Barplot of Contingent liabilities / Net worth (%)')
plt.show()

In [ ]:
FRA_df['Contingent_liabilities_by_Net_worth_in_Percentage'].groupby(FRA_df['Default']).mean()

In [ ]:
sns.barplot(data=FRA_df,x='Default',y='Net_working_capital')
plt.title('Barplot of Net working capital')
plt.show()

In [ ]:
corr=FRA_df.drop('Default',axis=1).corr()

In [ ]:
sns.heatmap(corr,xticklabels=corr.columns,yticklabels=corr.columns)
#sns.set(rc = {'figure.figsize':(30,40)})
plt.title('Correlation-Heatmap')
plt.show()

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X=FRA_df.drop(['Networth_Next_Year','Default'],axis=1)
y=FRA_df.pop('Default')

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=1)

In [ ]:
s_1=X_train.shape[0]
print('No.of records in the Train set is:{}'.format(s_1))

In [ ]:
s_2=X_test.shape[0]
print('No.of records in the Test set is:{}'.format(s_2))

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
vif_data = pd.DataFrame()
vif_data["Feature"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                          for i in range(len(X.columns))]
print(vif_data)

In [ ]:
vif_data_model=vif_data[vif_data['VIF']<=5]
vif_data_model

In [ ]:
print(y_train.value_counts(normalize = True).round(2))
print(' ')
print(y_test.value_counts(normalize = True).round(2))

In [ ]:
Default_train=pd.concat([X_train,y_train],axis=1)
Default_test=pd.concat([X_test,y_test],axis=1)

In [ ]:
Default_train.head()

In [ ]:
import statsmodels.formula.api as SM

In [ ]:
f_1='Default ~ PBDITA_to_total_income + PAT_to_net_worth + Total_capital + Contingent_liabilities_by_Net_worth_in_Percentage + Cash_to_current_liabilities_in_times + Cash_to_average_cost_of_sales_per_day + Creditors_turnover + Debtors_turnover + Finished_goods_turnover + WIP_turnover + Raw_material_turnover + Shares_outstanding + Equity_face_value + PE_on_BSE'

In [ ]:
model_1 = SM.logit(f_1, data= Default_train).fit()

In [ ]:
model_1.summary()

In [ ]:
f_2='Default ~ + PAT_to_net_worth + Contingent_liabilities_by_Net_worth_in_Percentage'

In [ ]:
model_2 = SM.logit(f_2, data= Default_train).fit()

In [ ]:
model_2.summary()

In [ ]:
y_prob_pred_train=model_2.predict(Default_train)

In [ ]:
y_prob_pred_train

In [ ]:
y_class_pred=[]
for i in range(0,len(y_prob_pred_train)):
    if np.array(y_prob_pred_train)[i]>0.5:
        a=1
    else:
        a=0
    y_class_pred.append(a)

In [ ]:
from sklearn import metrics

In [ ]:
sns.heatmap((metrics.confusion_matrix(Default_train['Default'],y_class_pred)),annot=True,cmap='Blues')
plt.xlabel('Predicted');
plt.ylabel('Actuals',rotation=0);

In [ ]:
from sklearn.metrics import roc_curve
fpr, tpr, thresholds = roc_curve(Default_train['Default'],y_prob_pred_train)

In [ ]:
optimal_idx = np.argmax(tpr-fpr)
optimal_threshold = thresholds[optimal_idx]
optimal_threshold.round(2)

In [ ]:
y_class_pred=[]
for i in range(0,len(y_prob_pred_train)):
    if np.array(y_prob_pred_train)[i]>0.22:
        a=1
    else:
        a=0
    y_class_pred.append(a)

In [ ]:
sns.heatmap((metrics.confusion_matrix(Default_train['Default'],y_class_pred)),annot=True,cmap='Blues')
plt.xlabel('Predicted');
plt.ylabel('Actuals',rotation=0);

In [ ]:
print(metrics.classification_report(Default_train['Default'],y_class_pred,digits=3))

In [ ]:
y_prob_pred_test=model_2.predict(Default_test)

In [ ]:
y_class_pred_test=[]
for i in range(0,len(y_prob_pred_test)):
    if np.array(y_prob_pred_test)[i]>0.22:
        a=1
    else:
        a=0
    y_class_pred_test.append(a)

In [ ]:
sns.heatmap((metrics.confusion_matrix(Default_test['Default'],y_class_pred_test)),annot=True,cmap='Blues')
plt.xlabel('Predicted');
plt.ylabel('Actuals',rotation=0);

In [ ]:
print(metrics.classification_report(Default_test['Default'],y_class_pred_test,digits=3))

In [ ]:
# Recall = TP/(TP+FN)
# Precission = TP/(TP+FP)

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

In [ ]:
lda=LinearDiscriminantAnalysis()

In [ ]:
lda_model=lda.fit(X_train,y_train)

In [ ]:
predict_train_lda=lda_model.predict(X_train)
predict_test_lda=lda_model.predict(X_test)

In [ ]:
print(metrics.classification_report(y_train,predict_train_lda))

In [ ]:
print(metrics.classification_report(y_test,predict_test_lda))

In [ ]:
predict_train_lda_prob=lda_model.predict_proba(X_train)[:,1]

In [ ]:
predict_test_lda_prob=lda_model.predict_proba(X_test)[:,1]

In [ ]:
fpr, tpr, thresholds = roc_curve(y_train,predict_train_lda_prob)

In [ ]:
optimal_idx = np.argmax(tpr-fpr)
optimal_threshold = thresholds[optimal_idx]
optimal_threshold.round(2)

In [ ]:
y_class_pred=[]
for i in range(0,len(predict_train_lda_prob)):
    if np.array(predict_train_lda_prob)[i]>0.22:
        a=1
    else:
        a=0
    y_class_pred.append(a)

In [ ]:
sns.heatmap((metrics.confusion_matrix(y_train,y_class_pred)),annot=True,cmap='Blues')
plt.xlabel('Predicted');
plt.ylabel('Actuals',rotation=0);

In [ ]:
print(metrics.classification_report(y_train,y_class_pred,digits=3))

In [ ]:
y_class_pred=[]
for i in range(0,len(predict_test_lda_prob)):
    if np.array(predict_test_lda_prob)[i]>0.23:
        a=1
    else:
        a=0
    y_class_pred.append(a)

In [ ]:
sns.heatmap((metrics.confusion_matrix(y_test,y_class_pred)),annot=True,cmap='Blues')
plt.xlabel('Predicted');
plt.ylabel('Actuals',rotation=0);

In [ ]:
print(metrics.classification_report(y_test,y_class_pred,digits=3))

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

In [ ]:
param_grid={
    'max_depth':[3,5,7],
    'min_samples_leaf':[5,10,15],
    'min_samples_split':[15,30,45],
    'n_estimators':[25,50]
}
rfcl=RandomForestClassifier()
grid_search=GridSearchCV(estimator=rfcl,param_grid=param_grid)

In [ ]:
grid_search.fit(X_train,y_train)

In [ ]:
grid_search.best_params_

In [ ]:
best_grid=grid_search.best_estimator_

In [ ]:
predict_train_rf=best_grid.predict(X_train)
predict_test_rf=best_grid.predict(X_test)

In [ ]:
print(metrics.classification_report(y_train,predict_train_rf,digits=3))

In [ ]:
print(metrics.classification_report(y_test,predict_test_rf,digits=3))